# RAG pipelines applied to Unipa's website

## Dependencies
- `haystack-ai` is the preview of Haystack 2.0
- `sentence_transformers` is needed for embeddings
- `transformers` is needed to use open-source LLMs
- `accelerate` and `bitsandbytes` are required to use quantized versions of these models (with smaller memory footprint)

In [1]:
%%capture
! pip install haystack-ai transformers accelerate bitsandbytes sentence_transformers

## Setup

In [2]:
from IPython.display import Image
from pprint import pprint
import torch
import rich
import random

In [42]:
!wget https://raw.githubusercontent.com/pierfrancescomartinello/NLP-Project/main/output/unipa_dataset.json

--2024-06-18 15:52:37--  https://raw.githubusercontent.com/pierfrancescomartinello/NLP-Project/main/output/unipa_dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7606379 (7.3M) [text/plain]
Saving to: ‘unipa_dataset.json’

unipa_dataset.json  100%[===================>]   7.25M  --.-KB/s    in 0.02s   

2024-06-18 15:52:37 (302 MB/s) - ‘unipa_dataset.json’ saved [7606379/7606379]



In [43]:
import pandas as pd
import numpy as np

filepath = "/content/unipa_dataset.json"
df = pd.read_json(filepath)
df.columns = ["title", "addr", "text"]

df

,title,addr,text
0,Università degli Studi di Palermo,https://www.unipa.it/,
1,Organi di Governo | Università degli Studi di ...,https://www.unipa.it/ateneo/OrganiDiGovernoECo...,
2,Fatturazione elettronica | Università degli St...,https://www.unipa.it/target/imprese/informazio...,Il D.M 55 del 3 aprile 2013 prevede l'obbligo ...
3,Presentazione | Università degli Studi di Palermo,https://www.unipa.it/ateneo/presentazione/,
4,Credits | Università degli Studi di Palermo,https://www.unipa.it/credits.html,I contenuti della home page e delle relative s...
...,...,...,...
6333,Calendari didattici DARCH | Centro per l’innov...,https://www.unipa.it/strutture/cimdu/Calendari...,Calendario didattico DARCH A.A. 2024/2025 Ca...
6334,| Università degli Studi di Palermo,https://www.unipa.it/amministrazione/rettorato...,
6335,Settore Comunicazione e URP | Settore Comunica...,https://www.unipa.it/amministrazione/rettorato...,
6336,Regolamenti per aree tematiche di interesse | ...,https://www.unipa.it/servizi/prevenzionedellac...,REGOLAMENTI PERSONALE DOCENTE E RICERCATORE


## Preprocessing

In [44]:
import re


def remove_dates(s: str) -> str:
    regex = r"\d{1,2}-(gen|feb|mar|apr|mag|giu|lug|ago|set|ott|nov|dic)-\d{4}"

    return re.sub(regex, "", s)


def remove_empty_docs(df):
    df["text"] = df["text"].replace("", np.nan)
    df.dropna(subset=["text"], inplace=True)

    return df


def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df["text"] = df["text"].apply(lambda x: x.strip())
    df = remove_empty_docs(df)
    df["text"] = df["text"].apply(remove_dates)

    # specify minimum document length
    # most documents below threshold tend to be garbage documents
    df = df[df["text"].str.len() >= 200]

    df.reset_index(inplace=True, drop=True)

    return df

In [45]:
df = preprocess(df)
df

,title,addr,text
0,Fatturazione elettronica | Università degli St...,https://www.unipa.it/target/imprese/informazio...,Il D.M 55 del 3 aprile 2013 prevede l'obbligo ...
1,Credits | Università degli Studi di Palermo,https://www.unipa.it/credits.html,I contenuti della home page e delle relative s...
2,Sostegno allo studio | Centro per l’innovazion...,https://www.unipa.it/strutture/cimdu/Sostegno-...,"Da 15 anni ItaStra, Scuola di Lingua Italian..."
3,PNRR | PNRR | Università degli Studi di Palermo,https://www.unipa.it/progetti/pnrr/,Il Piano Nazionale di Ripresa e Resilienza - P...
4,Corsi di preparazione alle prove di accesso A....,https://www.unipa.it/Corsi-di-preparazione-all...,Sono aperte le iscrizioni all’edizione inverna...
...,...,...,...
2774,U.O. Didattica e Internazionalizzazione - Desc...,https://www.unipa.it/strutture/cimdu/U.O.-Dida...,Il Responsabile dell’U.O. Didattica e Interna...
2775,Ricercatori neoassunti | Università degli Stud...,https://www.unipa.it/strutture/cimdu/ricercato...,Per restare sempre aggiornati è possiblie iscr...
2776,Declaratorie della U.O. | Centro per l’innovaz...,https://www.unipa.it/strutture/cimdu/Declarato...,La U.O. Didattica e Internazionalizzazione si...
2777,Calendari didattici DARCH | Centro per l’innov...,https://www.unipa.it/strutture/cimdu/Calendari...,Calendario didattico DARCH A.A. 2024/2025 Cal...


In [8]:
from haystack.dataclasses import Document

titles = list(df["title"].values)
texts = list(df["text"].values)
urls = list(df["addr"].values)

raw_docs = []
for title, text, url in zip(titles, texts, urls):
    raw_docs.append(Document(content=text, meta={"name": title or "", "url": url}))

## Indexing Pipeline

In [9]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import (
    SentenceTransformersTextEmbedder,
    SentenceTransformersDocumentEmbedder,
)
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

In [10]:
# vedere dopo
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

Our indexing Pipeline transform the original Documents and save them in the Document Store.

It consists of several components:

- `DocumentCleaner`: performs a basic cleaning of the Documents
- `DocumentSplitter`: chunks each Document into smaller pieces (more appropriate for semantic search and RAG)
- `SentenceTransformersDocumentEmbedder`:
  - represent each Document as a vector (capturing its meaning).
  - we choose a good but not too big model from [MTEB leaderboard](https://huggingface.co/spaces/mteb/leaderboard).
  - Also the metadata `title` is embedded, because it contains relevant information (`metadata_fields_to_embed` parameter).
  - We use the GPU for this expensive operation (`device` parameter).
- `DocumentWriter` just saves the Documents in the Document Store

In [11]:
indexing = Pipeline()
indexing.add_component("cleaner", DocumentCleaner())
indexing.add_component(
    "splitter", DocumentSplitter(split_by="sentence", split_length=2)
)
indexing.add_component(
    "doc_embedder",
    SentenceTransformersDocumentEmbedder(
        model="thenlper/gte-large",
        device=ComponentDevice.from_str("cuda:0"),
        meta_fields_to_embed=["title"],
    ),
)
indexing.add_component(
    "writer",
    DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE),
)

indexing.connect("cleaner", "splitter")
indexing.connect("splitter", "doc_embedder")
indexing.connect("doc_embedder", "writer")

🚅 Components
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - doc_embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - cleaner.documents -> splitter.documents (List[Document])
  - splitter.documents -> doc_embedder.documents (List[Document])
  - doc_embedder.documents -> writer.documents (List[Document])

In [13]:
indexing.run({"cleaner": {"documents": raw_docs}})

Batches:   0%|          | 0/948 [00:00<?, ?it/s]

{'writer': {'documents_written': 30317}}

Let's inspect the total number of chunked Documents and examine a Document

In [14]:
len(document_store.filter_documents())

27845

In [15]:
document_store.filter_documents()[0].meta

{'name': 'Fatturazione elettronica | Università degli Studi di Palermo',
 'url': 'https://www.unipa.it/target/imprese/informazioni/fatturazione-elettronica/',
 'source_id': '78eb0a1349eff5f5fda46f5d6efeeffc87096deee81660c984d1f79334123146',
 'page_number': 1}

In [16]:
pprint(document_store.filter_documents()[0])
print(len(document_store.filter_documents()[0].embedding))  # embedding size

Document(id=c81160b1c9bd873b1ae604c85641fd6e8290df7a920af385697caf26e8afec58, content: 'Il D.M 55 del 3 aprile 2013 prevede l'obbligo della fatturazione elettronica in tutti i rapporti con...', meta: {'name': 'Fatturazione elettronica | Università degli Studi di Palermo', 'url': 'https://www.unipa.it/target/imprese/informazioni/fatturazione-elettronica/', 'source_id': '78eb0a1349eff5f5fda46f5d6efeeffc87096deee81660c984d1f79334123146', 'page_number': 1}, embedding: vector of size 1024)
1024


## RAG Pipeline

### `HuggingFaceLocalGenerator` with `zephyr-7b-beta`

- To load and manage Open Source LLMs in Haystack 2.0, we can use the `HuggingFaceLocalGenerator`.

- The LLM we choose is [Zephyr 7B Beta](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), a fine-tuned version of Mistral 7B V.01 that focuses on helpfulness and outperforms many larger models on the MT-Bench and AlpacaEval benchmarks; the model was fine-tuned by the Hugging Face team.

- Since we are using a free Colab instance (with limited resources), we load the model using **4-bit quantization** (passing the appropriate `huggingface_pipeline_kwargs` to our Generator).
For an introduction to Quantization in Hugging Face Transformers, you can read [this simple blog post](https://huggingface.co/blog/merve/quantization).



In [17]:
from haystack.components.generators import HuggingFaceLocalGenerator

In [18]:
generator = HuggingFaceLocalGenerator(
    "HuggingFaceH4/zephyr-7b-beta",
    huggingface_pipeline_kwargs={
        "device_map": "auto",
        "model_kwargs": {
            "load_in_4bit": True,
            "bnb_4bit_use_double_quant": True,
            "bnb_4bit_quant_type": "nf4",
            "bnb_4bit_compute_dtype": torch.bfloat16,
        },
    },
    generation_kwargs={"max_new_tokens": 500},
)

Let's warm up the component and try the model...

In [19]:
generator.warm_up()

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

### `PromptBuilder`

 It's a component that renders a prompt from a template string using Jinja2 engine.

 Let's setup our prompt builder, with a format like the following (appropriate for Zephyr):

 `"<|system|>\nSYSTEM MESSAGE</s>\n<|user|>\nUSER MESSAGE</s>\n<|assistant|>\n"`

In [20]:
from haystack.components.builders import PromptBuilder

prompt_template = """<|system|>Using the information contained in the context, give a comprehensive answer to the question.
If the answer is contained in the context, also report the source URL.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
  {% for doc in documents %}
  {{ doc.content }} URL:{{ doc.meta['url'] }}
  {% endfor %};
  Question: {{query}}
  </s>
<|assistant|>
"""
prompt_builder = PromptBuilder(template=prompt_template)

### Pipeline Creation

In [21]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

Our RAG Pipeline finds Documents relevant to the user query and pass them to the LLM to generate a grounded answer.

It consists of several components:

- `SentenceTransformersTextEmbedder`: represent the query as a vector (capturing its meaning).
- `InMemoryEmbeddingRetriever`: finds the (top 5) Documents that are most similar to the query vector
- `PromptBuilder`
- `HuggingFaceLocalGenerator`

In [22]:
rag = Pipeline()
rag.add_component(
    "text_embedder",
    SentenceTransformersTextEmbedder(
        model="thenlper/gte-large", device=ComponentDevice.from_str("cuda:0")
    ),
)
rag.add_component(
    "retriever", InMemoryEmbeddingRetriever(document_store=document_store, top_k=5)
)
rag.add_component("prompt_builder", prompt_builder)
rag.add_component("llm", generator)

rag.connect("text_embedder", "retriever")
rag.connect("retriever.documents", "prompt_builder.documents")
rag.connect("prompt_builder.prompt", "llm.prompt")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: HuggingFaceLocalGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

## Questions

In [23]:
def get_generative_answer(query):

    results = rag.run(
        {"text_embedder": {"text": query}, "prompt_builder": {"query": query}}
    )

    answer = results["llm"]["replies"][0]
    rich.print(answer)

In [24]:
get_generative_answer(
    "What are the courses of Data, Algorithm and Machine Intelligence at Unipa?"
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The courses offered under the program Data, Algorithms, and Machine Intelligence at Unipa are:

1. Combinatorial and Probabilistic Algorithms
2. Big data management
3. Multisensory data exploration
4. Knowledge representation and reasoning
5. Cybersecurity
6. Information theory and data compression
7. Data encryption and codes
8. Cloud and high performance computing
9. Complex networks
10. Artificial intelligence and deep learning
11. Machine intelligence for optimization
12. Information retrieval and natural language processing
13. Pattern Discovery for Life Sciences

These courses are part of the study program LM 18: Data, Algorithms, and Machine Intelligence, which requires one 
of the following qualifications:

1. Italian Bachelor (Laurea) in L-31 scienze e tecnologie informatiche; L-35 scienze matematiche; L-8 ingegneria 
dell’informazione; L-41 statistica; L-30 scienze e tecnologie fisiche
2. Any Italian Bachelor (Laurea) or Master (Laurea Magistrale) including 12CFU (ECTS credits) of Mathematics and 
Physics, and 24 CFU of Computer and Information Sciences
3. Any foreign Degree that is equivalent to those described in points a) and b)

In addition, students are required to have an English language proficiency level of B2 on the Common European 
Framework of Reference for Languages. The source URL for this information is: 
https://www.unipa.it/dipartimenti/matematicaeinformatica/cds/dataalgorithmsandmachineintelligence2270/

In [25]:
get_generative_answer("Who were the deans of University of Palermo?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The provided context lists a comprehensive list of rectors, or university presidents, of the University of Palermo 
for various historical periods. However, it also includes a separate list of deans, or department heads, for the 
Department of Mathematics, Physics, and Computer Science (ME.PRE.C.C.) of the University of Palermo. The URL 
provided for this list is: https://www.unipa.it/dipartimenti/me.pre.c.c/

Additionally, there is a link provided for the legal department, Giurisprudenza, which includes a list of deans for
the years 1993-2008. The URL for this list is: 
https://www.unipa.it/dipartimenti/di.gi./cds/giurisprudenzasedetp471/didattica/tutorato.html

The context also includes a link to a biography of a former rector, Prof. Francesco Spallitta, who served as rector
from 1918-1921. The URL for this biography is: 
https://www.unipa.it/?lista=archivio&id=4eb4b438-d856-11e4-9500-005056010139

Lastly, the context mentions a joint project between the University of Palermo and AMAT Palermo S.p., but no 
specific information is provided about any deans or administrators involved in this collaboration.

Overall, the provided context includes lists of deans for specific departments and time periods, as well as a 
biography of a former rector. However, no comprehensive list of all deans throughout the history of the University 
of Palermo is provided.

In [26]:
get_generative_answer("Tell me about Data, Algorithms and Machine Intelligence")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Master's Degree program in Data, Algorithms, and Machine Intelligence, as described in the context, provides 
students with theoretical, experimental, and practical skills in data representation and computation for efficient 
and intelligent information processing. The program covers transversal topics in Data, Algorithms, and Machine 
Intelligence, as well as in-depth studies in application areas where data are big, relations are complex, and 
machine-learning structures are deep. The program requires either an Italian Bachelor (Laurea) in specific fields 
or a foreign degree equivalent to those listed, as well as English proficiency at the B2 level of the Common 
European Framework of Reference for Languages. Some specific courses offered as part of this program include 
Artificial Intelligence and Deep Learning, Big Data & Analytics, Game Theory, Financial Planning and Control, and 
Categorical Data. This program is part of the LM 18 study program, and more information can be found on the 
websites provided. Additionally, there is an upcoming seminar called "AI 4 FUTURE" that will provide insights into 
harnessing the power of data and understanding its impact on innovation.

In [27]:
get_generative_answer("When was Unipa founded?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The context provided does not contain information about when Unipa was founded. Therefore, an answer cannot be 
deduced from the context.

In [32]:
get_generative_answer("Tell me about the Ingegneria Informatica degree at University of Palermo")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The University of Palermo offers a Laurea Magistrale in Informatica that also allows students to register in the 
Engineers' Register (Section A). In addition to graduates in Informatics (class L31), graduates in Ingegneria 
Informatica and Telecomunicazioni, Ingegneria Gestionale e Informatica, and Ingegneria Cibernetica (class L8) can 
also enroll in this program, as well as graduates in Mathematics (class L35) and Physics (class L30) with 
appropriate course selections. This program was accredited by GRIN (GRuppo dei professori e ricercatori in 
INformatica) in 2015, making it the only master's program in Informatics at the University of Palermo with this 
certification. The curriculum covers advanced topics in computer science, such as Information Theory and Computing 
Models, as well as specialized areas, including advanced algorithm design, data management and analysis, 
infrastructure and network design, with a focus on service quality, security, and data privacy. Graduates from 
related fields, such as Law, Engineering, Mathematics, Statistics, and Economics, are invited to attend an online 
Engineering Ingegneria Informatica Recruiting Day, where they will learn about the job profiles and employment 
opportunities available to them, including positions for junior Business Analysts, Software Developers, and 
Consultants.

In [47]:
get_generative_answer("Cos'è phishing?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Phishing è una truffa informatica che ha come obiettivo il furto illegittimo delle credenziali di accesso a un 
account riservato, in questo caso, gli account @unipa.it. I truffatori tentano di indirizzare l'utente su un sito 
falso mediante scuse come la scadenza dell'account o la segnalazione di errori da correggere urgentemente. Per 
proteggersi dal phishing, si consiglia di seguire due regole basilari: se non si è sicuri della validità di un 
dominio, non utilizzare le proprie credenziali, e se si sono già inserite le proprie credenziali, cambiare 
immediatamente la propria password. (Fonte: https://www.unipa.it/phishing.html)

In [46]:
get_generative_answer("What's phishing?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Phishing is a type of cybercrime where perpetrators attempt to obtain unauthorized access to an account by tricking
the user into providing their password. This is typically done by sending fraudulent emails or messages that appear
to be from a legitimate source, such as a bank or a service provider, and urging the user to take immediate action,
such as updating their password or verifying their account information. The user is then directed to a fake website
or prompted to provide their login credentials, which are then used to gain unauthorized access to their account. 
To protect oneself from phishing scams, it's recommended to only use trusted websites and to avoid providing 
sensitive information unless absolutely necessary. If there is any doubt about the legitimacy of a message or 
website, it's best to err on the side of caution and not provide any personal information. For more information on 
phishing and how to protect oneself, you can refer to the resources provided in the context, such as the links to 
the University of Rome "La Sapienza" website on phishing prevention and election login pages.